# **Day 4: Scratchcards**
This one seems pretty simple at first glance! Shouldn't be too tough. 

# Setup
The cells below will set up the rest of the notebook. 

I'll start by configuring my kernel:

In [1]:
# Changing the current working directory
%cd ..

# Enabling the autoreload extension
%load_ext autoreload
%autoreload 2

/Users/thubbard/Documents/Personal/Programming/advent-of-code-2023


Now, I'm going to import some libraries:

In [2]:
# Import statements
import pandas as pd
import re

Finally, I'll load in the data for this puzzle. 

In [4]:
# Load in the data for the puzzle
with open("data/input-files/day-04-input.txt", "r") as txt_file:
    input_data = txt_file.readlines()

# Identifying Winning Numbers
Actually identifying the winning numbers in each of these games seems pretty trivial. I have a feeling the numbers themselves are going to come back in Part 2, so I'll keep track of which numbers are actually winning. 

In [20]:
def parse_input_into_dataframe(input_data):
    """
    This method will parse the puzzle's input_data into a DataFrame that contains various 
    pieces of info about the game. 
    """
    
    # Parse the ID, winning numbers, and ticket numbers from each of the lines into a DataFrame
    scratchcards_df = pd.DataFrame.from_records(
        [
            {
                "card_number": int(line.split(":")[0].split(" ")[-1]),
                "winning_numbers": [
                    int(x.strip())
                    for x in line.split(":")[-1].strip().split("|")[0].strip().split(" ")
                    if x != ""
                ],
                "ticket_numbers": [
                    int(x.strip())
                    for x in line.split(":")[-1].strip().split("|")[-1].strip().split(" ")
                    if x != ""
                ],
            }
            for line in input_data
        ]
    )

    # Determine which ticket_numbers are winning_numbers
    scratchcards_df["winning_ticket_numbers"] = scratchcards_df.apply(
        lambda row: [
            ticket_num
            for ticket_num in row.ticket_numbers
            if ticket_num in row.winning_numbers
        ],
        axis=1,
    )

    # Determine the point value of each of the cards
    scratchcards_df["point_value"] = scratchcards_df["winning_ticket_numbers"].apply(
        lambda winning_numbers_list: (2 ** (len(winning_numbers_list) - 1))
        if len(winning_numbers_list) >= 1
        else 0
    )
    
    # Return the DataFrame
    return scratchcards_df

With this method in hand, we can calculate the sum of the values of the Elf's cards:

In [24]:
# Parse the input_data using the method we'd created
scratchcards_df = parse_input_into_dataframe(input_data)

# Calculate the sum of the point_values, and then print it
point_value_sum = scratchcards_df["point_value"].sum()
print(f"The sum of the values of the scratchcards is '{point_value_sum}'")

The sum of the values of the scratchcards is '25010'
